In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.ticker as ticker
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import os 
import time 
from itertools import product

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
%matplolib inline

UsageError: Line magic function `%matplolib` not found.
